# CLSmith Results

To generate results:

```
# make programs
$ ./mkprograms -H cc1 -n 10000
# list available OpenCL platforms and devices
$ ./clinfo
# run programs on platform ID and device ID
$ ./runprograms -H cc1 0 0
```

In [1]:
import db
hostname = "cc1"
db.init(hostname)

In [2]:
with db.Session() as session:
    nprog = session.query(db.Program).count()
    nparam = session.query(db.Params).count()
    ntestbed = session.query(db.Testbed).count()
    nresult = session.query(db.Result).count()
    print("#. programs:", nprog)
    print("#. params", nparam)
    print("#. testbeds:", ntestbed)
    print("#. results:", nresult)
    print()
    print("Testbeds:")
    for testbed in session.query(db.Testbed):
        nresult_tb = session.query(db.Result).filter(db.Result.testbed_id == testbed.id).count()
        ratio_result = nresult_tb / nresult
        testbed_str = ("  id[{testbed.id}] {testbed.hostname} {testbed.device:.16}, "
                       "{nresult_tb} results ({ratio_result:.1%} of total)").format(**vars())
        print(testbed_str)

#. programs: 2799
#. params 1
#. testbeds: 2
#. results: 1422

Testbeds:
  id[1] cc1 GeForce GTX 1080, 669 results (47.0% of total)
  id[5] cc3 Intel(R) Xeon(R), 753 results (53.0% of total)


In [3]:
with db.Session() as session:
    for param in session.query(db.Params).all():
        suspicious = []

        print("Params:", param)
        programs_with_results = session.query(db.Program).filter(
            db.Program.id.in_(session.query(db.Result.program_id).filter(db.Result.params_id == param.id)))
        programs_with_good_results = session.query(db.Program).filter(
            db.Program.id.in_(session.query(db.Result.program_id).filter(
                db.Result.params_id == param.id, db.Result.status == 0)))
        ratio_good = programs_with_good_results.count() / programs_with_results.count()

        print("  #. programs with results", programs_with_results.count())
        print("  #. programs with good results", programs_with_good_results.count(),
              "({ratio_good:.1%} of total)".format(**vars()))
        
        for program in programs_with_good_results.all():
            q = session.query(db.Result).filter(
                    db.Result.program_id == program.id, db.Result.params_id == param.id)
            
            stdout = set(r.stdout for r in q.all())
            if len(stdout) > 1:
                suspicious.append(q.first().program_id)
        
        nsuspicious = len(suspicious)
        ratio_suspicious = nsuspicious / programs_with_good_results.count()
        print("  #. suspicious {nsuspicious} ({ratio_suspicious:.1%} of total)".format(**vars()))

Params: global: (128, 16, 1), local: (32, 1, 1)
  #. programs with results 753
  #. programs with good results 610 (81.0% of total)
  #. suspicious 95 (15.6% of total)
